<a href="https://colab.research.google.com/github/baoloc97/Data-science/blob/main/Image_Classfication_BanProduct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Những điều cần làm trước khi chạy
# install "pandas",
#         "numpy",
#         "matplotlib",
#         "openpyxl",
#         "opencv-python",
#         "keras",
#         "absl-py 0.12.0"
#         "pickle-mixin 1.0.2"

# tạo thư mục train có 2 thư mục nhỏ là ban và unban
# tạo thư mục test có 3 thư mục nhỏ là ban , unban và predict
# Chạy file Preprocessing.py trước để preprocessing data trong thư mục train sau đó đưa về ma trận và đóng gói dữ liệu = pickle
# Chạy xong Preprocessing.py ta sẽ có 2 file X.pkl và y.pkl, X.pkl chứa dữ liệu hình và y.pkl chứa label của hình
# Tiếp theo chạy file Train.py để train model. Sau khi train xong sẽ có  model 3x3x64-banvsunban.model
# Chạy file predict.py. Thì sẽ lây hình ở test/ban đánh số 0 và test/unban đánh số 1. sau đó suffer lên rồi đổ hết hình qua test/predict
# Sau khi predict xong thì ta sẽ có được accuracy của model do so sánh với dữ liệu ta đã đánh số từ trước.
# File Final_predict.py dùng để predict 1 hình input là đường dẫn của hình output là 1 dictionary {'ban': 6.488902e-17, 'unban': 1.0}

In [ ]:
!unzip Ban_product.zip -d 'train/ban/'

In [ ]:
!unzip Unban_product.zip -d 'train/unban/'

In [ ]:
!unzip Test_ban.zip -d 'test/ban/'

In [ ]:
!unzip Test_unban.zip -d 'test/unban/'

In [ ]:
!unzip Test.zip -d 'test/predict'

In [22]:
import numpy as np 
import cv2
import os
import random
import matplotlib.pyplot as plt 
import pickle


In [23]:
DIRECTORY = r'train'
CATEGORIES =  ['ban','unban']


In [24]:
IMG_SIZE = 60
data = []

for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        label = CATEGORIES.index(category)
        arr = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        new_arr = cv2.resize(arr, (IMG_SIZE, IMG_SIZE))
        data.append([new_arr, label])

In [25]:
random.shuffle(data)

In [26]:
X = []
y = []
for features, labels in data:
	X.append(features)
	y.append(labels)

X = np.array(X)
y = np.array(y)

print(X[0])

[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 204 204 205]
 [255 255 255 ... 204 204 205]
 [255 255 255 ... 252 202 204]]


In [27]:
pickle.dump(X,open('X.pkl','wb'))
pickle.dump(y,open('y.pkl','wb'))

In [28]:
import pickle
import keras
from keras import models
from keras.models import Sequential
from keras.layers import Conv2D,MaxPool2D,Flatten,Dense,Dropout
import keras.losses
keras.losses.custom_loss = 'spare_categorical_crossentrop'
X = pickle.load(open('X.pkl','rb'))
y = pickle.load(open('y.pkl','rb'))

IMG_SIZE = 60 

In [29]:
X = X/255


In [ ]:
#y = np.asarray(y).astype('float32').reshape((1,-1))

In [30]:
print(X.shape)
print(y.shape)

(142, 60, 60)
(142,)


In [31]:
X = X.reshape(-1, IMG_SIZE, IMG_SIZE, 1) #img_size phải trùng với img_size lúc preprocessing

In [67]:
#Ở đây Dùng CNN ~ 64%, google net, effecient net work better ~ 76%
model = Sequential()

model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(MaxPool2D((2,2)))

model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(MaxPool2D((2,2)))

model.add(Flatten())

model.add(Dense(128, input_shape = X.shape[1:], activation = 'relu'))
model.add(Dropout(0.4))
model.add(Dense(2, activation = 'softmax'))

In [69]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X, y, epochs=10, validation_split=0.2,batch_size=32)

In [43]:
import time # tính thời gian train

time.time()

1626081247.6913338

In [44]:
model.save('3x3x64-banvsunban.model')

INFO:tensorflow:Assets written to: 3x3x64-banvsunban.model/assets


In [ ]:
!unzip Test.zip -d 'test/predict/'

In [ ]:
!unzip unbantest.zip -d 'test/unban/'

In [47]:
# mình làm predict ở đây nha
import cv2
import keras
import numpy as np
from keras.preprocessing import image
from os import listdir
from os.path import isfile, join


CATEGORIES = ['ban', 'unban']


def image(path):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    new_arr = cv2.resize(img, (60, 60))
    new_arr = np.array(new_arr)
    new_arr = new_arr.reshape(-1, 60, 60, 1)
    return new_arr

model = keras.models.load_model('3x3x64-banvsunban.model')



In [79]:
path ='test/predict/11.jpg'
try:
  picture = [image(path)]
except Exception as e:
  print('Image Error')

prediction = model.predict(picture)
print(CATEGORIES[prediction.argmax()])

Final_result = {
  "ban": prediction[0][0],
  "unban": prediction[0][1]
}
print(Final_result) 

unban
{'ban': 6.488902e-17, 'unban': 1.0}


In [48]:
DIRECTORY_Test = r'test'
CATEGORIES_Test =  ['ban','unban']

In [52]:
data_test = []
for category in CATEGORIES_Test:
    path = os.path.join(DIRECTORY_Test, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        label = CATEGORIES_Test.index(category)

        data_test.append([img,label])
random.shuffle(data_test)       
print(data_test[:3])  
print(len(data_test))
for i in range(len(data_test)-1000):
  print(data_test[i][1])      

[['979.jpg', 0], ['24.jpg', 0], ['34.jpg', 1]]
142


In [ ]:
!unzip bantest.zip -d 'test/predict/'

In [ ]:
!unzip unbantest.zip -d 'test/predict/'

In [ ]:
data_test_size = len(data_test) 
#data_test_size = data_test_size - 980
acc =0 
mypath = "test/predict/"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
print(onlyfiles)
for i in range(len(data_test)):
  link = join(mypath,data_test[i][0])
  prediction = model.predict([image(link)])
  for i in range(len([image(link)])):
	  print("X=%s, Predicted=%s" % ([image(link)][i], prediction[i]))
  if prediction.argmax() == data_test[i][1]:
    acc = acc+1
    #print(prediction.argmax())
    #print(data_test[i][1])
  #print(CATEGORIES_Test[prediction.argmax()])

print("Predict Accuracy = ", 100*float(acc/data_test_size))  

In [ ]:
ynew = model.predict_classes(Xnew)
# show the inputs and predicted outputs
for i in range(len(Xnew)):
	print("X=%s, Predicted=%s" % (Xnew[i], ynew[i]))

In [83]:
rm -rf 3x3x64-banvsunban.model

In [ ]:
!python3 Finalpredict.py


In [84]:
!python3 Preprocess.py

[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]


In [85]:
!python3 Train.py

2021-07-12 10:51:39.392414: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
(142, 60, 60)
(142,)
2021-07-12 10:51:41.225502: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-07-12 10:51:41.236818: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-07-12 10:51:41.236874: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (669d1ebe3f96): /proc/driver/nvidia/version does not exist
2021-07-12 10:51:41.776535: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-07-12 10:51:41.777112: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2299995000 Hz
4/4 [==============================] - 23s 378ms/step -

In [86]:
!python3 Finalpredict.py

2021-07-12 10:53:00.460766: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-07-12 10:53:02.343745: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-07-12 10:53:02.355682: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-07-12 10:53:02.355738: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (669d1ebe3f96): /proc/driver/nvidia/version does not exist
2021-07-12 10:53:03.279580: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-07-12 10:53:03.280025: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2299995000 Hz
{'ban': 0.0, 'unban': 1.0}


In [87]:
!python3 Predict.py

2021-07-12 10:53:46.050994: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-07-12 10:53:47.907528: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-07-12 10:53:47.919641: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-07-12 10:53:47.919698: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (669d1ebe3f96): /proc/driver/nvidia/version does not exist
2021-07-12 10:53:48.885809: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-07-12 10:53:48.886305: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2299995000 Hz
Predict Accuracy =  50.0
So hinh doc duoc  71
So hinh ko doc duoc 0
